# COSA Flood Exploration

## Imports

In [1]:
import wrangle as w
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
pd.__version__

'1.1.3'

In [3]:
flood = clean_flood()

In [4]:
flood.head()

,datetime,sensor_to_water_feet,sensor_to_water_meters,sensor_to_ground_feet,sensor_to_ground_meters,flood_depth_feet,flood_depth_meters,flood_alert
0,2021-05-20 00:11:55,13.188976,4.02,13.500656,4.115,0.311680,0.095,No Risk
1,2021-05-20 00:26:55,13.254593,4.04,13.500656,4.115,0.246063,0.075,No Risk
2,2021-05-20 00:41:55,13.221785,4.03,13.500656,4.115,0.278871,0.085,No Risk
3,2021-05-20 00:56:55,13.221785,4.03,13.500656,4.115,0.278871,0.085,No Risk
4,2021-05-20 01:11:55,13.221785,4.03,13.500656,4.115,0.278871,0.085,No Risk


In [5]:
#flood.to_csv('bestflood.csv')

In [6]:
flood.sensor_to_ground_feet.value_counts()

13.500656    19995
13.500656     7559
Name: sensor_to_ground_feet, dtype: int64

In [7]:
flood.describe()

,sensor_to_water_feet,sensor_to_water_meters,sensor_to_ground_feet,sensor_to_ground_meters,flood_depth_feet,flood_depth_meters
count,27554.000000,27554.000000,2.755400e+04,2.755400e+04,27554.000000,27554.000000
mean,-74.586608,-77.202493,1.350066e+01,4.115000e+00,88.087264,81.317493
std,269.734840,268.919588,9.883353e-12,1.826128e-12,269.734840,268.919588
min,-999.000000,-999.000000,1.350066e+01,4.115000e+00,0.016404,0.005000
25%,0.225000,0.068600,1.350066e+01,4.115000e+00,0.639764,0.195000
50%,0.275000,0.083800,1.350066e+01,4.115000e+00,13.225656,4.031200
75%,12.860892,3.920000,1.350066e+01,4.115000e+00,13.275656,4.046400
max,13.484252,4.110000,1.350066e+01,4.115000e+00,1012.500656,1003.115000


In [8]:
flood.flood_depth_feet

0         0.311680
1         0.246063
2         0.278871
3         0.278871
4         0.278871
           ...    
27549    13.283956
27550    13.283956
27551    13.267356
27552    13.250656
27553    13.242356
Name: flood_depth_feet, Length: 27554, dtype: float64

In [2]:
#Fix some errors
# Flood Cleaning
def clean_flood():
    '''Drops unneeded columns from the med center flooding df
    Makes sure DateTime is in DateTime format'''
    # read csv
    flood = pd.read_csv('med_center_flood.csv')
    # drop the colums
    flood = flood.drop(columns=['LAT', 'LONG', 'Zone', 
                          'Sensor_id', 'SensorModel', 
                          'SensorStatus', 'AlertTriggered', 
                          'Temp_C', 'Temp_F', 'Vendor'])
    # Set to date time format
    flood.DateTime = pd.to_datetime(flood.DateTime)
    flood = flood.rename(columns={"DateTime": "datetime", 
                        "DistToWL_ft": "sensor_to_water_feet", 
                        "DistToWL_m": "sensor_to_water_meters", 
                        "DistToDF_ft": "sensor_to_ground_feet",
                        "DistToDF_m": "sensor_to_ground_meters"})
    # replae -999 with 0
    flood["sensor_to_ground_feet"].replace({-999:13.5006561680}, inplace=True)
    flood["sensor_to_ground_meters"].replace({-999:4.115}, inplace=True)
    
    #flood = flood.replace(to_replace=-999, value=0)
    # create new features for flood depth
    flood['flood_depth_feet'] = flood.sensor_to_ground_feet - flood.sensor_to_water_feet
    flood['flood_depth_meters'] = flood.sensor_to_ground_meters - flood.sensor_to_water_meters 
    # Create new alert
    def flood_alert(c):
        if 0 < c['flood_depth_feet'] < 10:
            return 'No Risk'
        elif 10 < c['flood_depth_feet'] < 11:
            return 'Minor Risk'
        elif 11 < c['flood_depth_feet'] < 12:
            return 'Moderate Risk'
        elif 12 < c['flood_depth_feet']:
            return 'Major Risk !'
        else:
            return 'No Alert'
    flood['flood_alert'] = flood.apply(flood_alert, axis=1)
    # return new df
    return flood

In [9]:
nines = flood[(flood.sensor_to_water_feet == -999)]

In [10]:
nines.shape

(2161, 8)

In [11]:
nines.head()

,datetime,sensor_to_water_feet,sensor_to_water_meters,sensor_to_ground_feet,sensor_to_ground_meters,flood_depth_feet,flood_depth_meters,flood_alert
5750,2021-05-13 00:00:00,-999.0,-999.0,13.500656,4.115,1012.500656,1003.115,Major Risk !
5751,2021-05-13 00:05:00,-999.0,-999.0,13.500656,4.115,1012.500656,1003.115,Major Risk !
5752,2021-05-13 00:10:00,-999.0,-999.0,13.500656,4.115,1012.500656,1003.115,Major Risk !
5753,2021-05-13 00:15:00,-999.0,-999.0,13.500656,4.115,1012.500656,1003.115,Major Risk !
5754,2021-05-13 00:20:00,-999.0,-999.0,13.500656,4.115,1012.500656,1003.115,Major Risk !


In [12]:
flood.shape

(27554, 8)

### Tableau Link